In [1]:
import math
import geopandas as gpd
import osmnx as ox
import pandas as pd
from shapely.geometry import LineString, Point

In [2]:
gdf = gpd.read_file("boston_streets_init_1")
gdf['length'] = gdf.geometry.length

In [3]:
gdf.crs = "EPSG:26919"

In [4]:
gdf = gdf.to_crs("epsg:4326")

In [5]:
gdf['coords'] = gdf.geometry.map(lambda x : x.coords)
gdf['coord_pairs'] = gdf.coords.map(lambda xs : list(zip(xs,xs[1:])))
gdf['linestrings'] = gdf.coord_pairs.map(lambda ls : [LineString(list(l)) for l in ls])

In [ ]:
sum(gdf.linestrings.map(lambda x : len(x)))

In [6]:
df_temp = pd.DataFrame(gdf)
df_temp = df_temp.drop(columns=['geometry','coords','coord_pairs']).explode('linestrings')
gdf = gpd.GeoDataFrame(df_temp, geometry='linestrings')

In [7]:
gdf.crs = "epsg:4326"
gdf['length'] = gdf.to_crs("EPSG:26919").length

In [8]:
gdf['ratio'] = gdf.length/gdf['length']

In [9]:
    gdf['ratio'].describe()

count    114724.000000
mean          0.000011
std           0.000001
min           0.000009
25%           0.000010
50%           0.000011
75%           0.000012
max           0.000012
Name: ratio, dtype: float64

What we see is that each meter is roughly 0.000011 degrees with very little deviation!

In [10]:
CUT_LENGTH_METERS = 8
CUT_LENGTH = 0.000011 * CUT_LENGTH_METERS
sum((gdf.length/CUT_LENGTH).map(math.ceil))

425130

In [11]:
# Thank you Sean Gilles https://gist.github.com/sgillies/465156#file_cut.py

def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]
        

# here's me
def recursive_cut(line, distance):
    if [line] == cut(line, distance):
        return [line]
    else:
        segment, rest = cut(line, distance)
        return [segment] + recursive_cut(rest, distance)
    
    

In [14]:
# DO NOT RECURSIVE_CUT THE STREETS
# SIDEWALKIFY THEM
# THEN RECURSIVE_CUT THE SIDEWALKS
gdf.to_crs("EPSG:26919").to_file("63_streets_to_sidewalkify.shp")

In [13]:
import sidewalkify
sidewalks = sidewalkify.graph.graph_workflow(gdf)

AttributeError: ("'Series' object has no attribute 'geometry'", 'occurred at index 0')

In [ ]:
df_temp = df_temp.explode('segments').drop(columns=['linestrings'])
gdf = gpd.GeoDataFrame(df_temp, geometry=df_temp.segments)

In [ ]:
(gdf.length/CUT_LENGTH).hist()

In [ ]:
gdf = gdf.drop(columns='segments')

In [ ]:
import math
def linestring_start(linestring):
    (l1,l2) = list(linestring.coords)[0]
    return (l2,l1)

def linestring_end(linestring):
    (l1,l2) = list(linestring.coords)[1]
    return (l2,l1)

def linestring_heading(linestring):
    # 0 is true north, 90 is east
    # so heading = 90 - usual_angle (in standard form)
    (l1,l2) = linestring_start(linestring)
    (m1,m2) = linestring_end(linestring)
    angle_deg = math.atan2(m2-l2, m1-l1) * 360 / (2 * math.pi)
    heading_deg = int((90 - angle_deg) % 360)
    return(heading_deg)
    
gdf['heading'] = gdf.geometry.map(linestring_heading)

In [ ]:
gdf

In [ ]:
from shapely import ops, geometry
def recreate_map(gdf):
    pass
    # NO
    # just have to check if we can sidewalkify this
    # also need to work out cross-walkify
    
# now feed into https://github.com/MKuranowski/pyroutelib3
# use OSM for geocoding I guess
# need to have crosswalks or nothing will happen :(

In [ ]:
import sidewalkify

In [ ]:
gdf.crs = "epsg:4326"
gdf_26919 = gdf.to_crs("epsg:26919")

In [ ]:
gdf_26919.to_file("segmented_streets_26919.shp")